In [ ]:
!pip install torchvision

# Загрузка Pascal VOC 2012
from torchvision import datasets
voc_dataset = datasets.VOCDetection(root='/path/to/pascal_voc', year='2012', image_set='trainval', download=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 50.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

100%|██████████| 2.00G/2.00G [01:45<00:00, 19.0MB/s]


### YOLOv8 требует аннотации в текстовом формате (.txt), а Pascal VOC предоставляет аннотации в формате XML. Нужно конвертировать XML в YOLO-формат, где каждая строка в .txt файле соответствует объекту в формате: <class_id> <x_center> <y_center> <width> <height> - нормализованных координат.

In [2]:
import os
import xml.etree.ElementTree as ET
from PIL import Image

voc_classes = [
    'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow',
    'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'
]

# Путь к данным
voc_root = '/path/to/pascal_voc/VOCdevkit/VOC2012'
annotations_dir = os.path.join(voc_root, 'Annotations')
images_dir = os.path.join(voc_root, 'JPEGImages')
output_dir = '/path/to/pascal_voc/yolo_labels'  # Папка для YOLO-аннотаций

# Создание папки для YOLO-аннотаций
os.makedirs(output_dir, exist_ok=True)

# Конвертация XML в YOLO
for xml_file in os.listdir(annotations_dir):
    if not xml_file.endswith('.xml'):
        continue

    # Парсинг XML
    xml_path = os.path.join(annotations_dir, xml_file)
    tree = ET.parse(xml_path)
    root = tree.getroot()

    # Получение размеров изображения
    img_name = root.find('filename').text
    img_path = os.path.join(images_dir, img_name)
    img = Image.open(img_path)
    img_width, img_height = img.size

    # Создание YOLO txt файла
    txt_name = xml_file.replace('.xml', '.txt')
    txt_path = os.path.join(output_dir, txt_name)

    with open(txt_path, 'w') as f:
        for obj in root.findall('object'):
            class_name = obj.find('name').text
            if class_name not in voc_classes:
                continue
            class_id = voc_classes.index(class_name)

            # Получение координат bounding box
            bndbox = obj.find('bndbox')
            xmin = float(bndbox.find('xmin').text)
            ymin = float(bndbox.find('ymin').text)
            xmax = float(bndbox.find('xmax').text)
            ymax = float(bndbox.find('ymax').text)

            # Конвертация в YOLO-формат
            x_center = (xmin + xmax) / 2 / img_width
            y_center = (ymin + ymax) / 2 / img_height
            width = (xmax - xmin) / img_width
            height = (ymax - ymin) / img_height

            # Запись в файл
            f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

### Pascal VOC предоставляет файлы в ImageSets/Main/ для разделения на тренировочную и валидационную выборки (train.txt, val.txt).

In [ ]:
import shutil

image_sets_dir = os.path.join(voc_root, 'ImageSets', 'Main')
train_list = os.path.join(image_sets_dir, 'train.txt')
val_list = os.path.join(image_sets_dir, 'val.txt')
images_train_dir = '/path/to/pascal_voc/images/train'
images_val_dir = '/path/to/pascal_voc/images/val'
labels_train_dir = '/path/to/pascal_voc/labels/train'
labels_val_dir = '/path/to/pascal_voc/labels/val'

os.makedirs(images_train_dir, exist_ok=True)
os.makedirs(images_val_dir, exist_ok=True)
os.makedirs(labels_train_dir, exist_ok=True)
os.makedirs(labels_val_dir, exist_ok=True)

with open(train_list, 'r') as f:
    for line in f:
        img_name = line.strip() + '.jpg'
        label_name = line.strip() + '.txt'
        shutil.copy(os.path.join(images_dir, img_name), os.path.join(images_train_dir, img_name))
        shutil.copy(os.path.join(output_dir, label_name), os.path.join(labels_train_dir, label_name))

with open(val_list, 'r') as f:
    for line in f:
        img_name = line.strip() + '.jpg'
        label_name = line.strip() + '.txt'
        shutil.copy(os.path.join(images_dir, img_name), os.path.join(images_val_dir, img_name))
        shutil.copy(os.path.join(output_dir, label_name), os.path.join(labels_val_dir, label_name))

In [4]:
!touch /path/to/pascal_voc/data.yaml

In [ ]:
data_yaml = '/path/to/pascal_voc/data.yaml'

# Проверка содержимого data.yaml
with open(data_yaml, 'r') as f:
    print(f.read())

In [6]:
import os

# Проверка наличия файлов
print(f"Train images: {len(os.listdir('/path/to/pascal_voc/images/train'))}")
print(f"Train labels: {len(os.listdir('/path/to/pascal_voc/labels/train'))}")
print(f"Val images: {len(os.listdir('/path/to/pascal_voc/images/val'))}")
print(f"Val labels: {len(os.listdir('/path/to/pascal_voc/labels/val'))}")

Train images: 5717
Train labels: 5717
Val images: 5823
Val labels: 5823


In [ ]:
data_yaml = '/path/to/pascal_voc/data.yaml'

### Сначала установим библиотеку Ultralytics и загрузим модель YOLOv8n (nano-версия для экономии ресурсов)

In [ ]:
!pip install ultralytics

# Импорт библиотек
import torch
from ultralytics import YOLO

# Проверка доступности GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Используемое устройство: {device}")

# Загрузка модели YOLOv11
model = YOLO('yolov8n.pt')  # Используем nano-версию для быстрого обучения

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.5 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Используемое устройство: cuda


100%|██████████| 6.25M/6.25M [00:00<00:00, 319MB/s]


### Обучим модель YOLOv8n на выбранном наборе данных. Количество эпох поставим значение 20, размер батча 16, размер изображения 640.

In [ ]:
# Обучение модели
results = model.train(
    data=data_yaml,
    epochs=20,
    batch=16,
    imgsz=640,
    device=device    # Устройство для обучения
)

Ultralytics 8.3.134 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/path/to/pascal_voc/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=

train: Scanning /path/to/pascal_voc/labels/train.cache... 5717 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5717/5717 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1353.4±910.9 MB/s, size: 99.9 KB)


val: Scanning /path/to/pascal_voc/labels/val.cache... 5823 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5823/5823 [00:00<?, ?it/s]


Plotting labels to runs/detect/train4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000417, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train4
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       5823      15787      0.779       0.65      0.723      0.527

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      20/20      3.34G     0.8226     0.9271       1.14          8        640: 100%|██████████| 358/358 [01:33<00:00,  3.81it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 182/182 [00:46<00:00,  3.94it/s]


                   all       5823      15787      0.781      0.653      0.727      0.531

20 epochs completed in 0.810 hours.
Optimizer stripped from runs/detect/train4/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train4/weights/best.pt, 6.2MB

Validating runs/detect/train4/weights/best.pt...
Ultralytics 8.3.134 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,009,548 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 182/182 [00:46<00:00,  3.88it/s]


                   all       5823      15787      0.781      0.653      0.727      0.532
             aeroplane        348        484        0.9      0.715      0.797      0.603
               bicycle        290        380      0.844       0.71      0.789      0.588
                  bird        374        629      0.781      0.598      0.689      0.478
                  boat        252        491      0.707      0.462      0.572      0.344
                bottle        369        733      0.776       0.48      0.576      0.388
                   bus        211        320      0.847      0.781      0.816      0.676
                   car        608       1173       0.82      0.663      0.761      0.551
                   cat        544        618      0.836      0.788      0.868      0.685
                 chair        642       1449      0.651      0.509       0.59      0.407
                   cow        154        347      0.737      0.695      0.723      0.523
           diningtabl

### После обучения оцениваем модель на валидационной выборке по значению mAP.

In [ ]:
metrics = model.val(data=data_yaml)

print(f"mAP50: {metrics.box.map50}")
print(f"mAP50-95: {metrics.box.map}")

Ultralytics 8.3.134 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2003.1±848.2 MB/s, size: 101.3 KB)


val: Scanning /path/to/pascal_voc/labels/val.cache... 5823 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5823/5823 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 364/364 [00:55<00:00,  6.62it/s]


                   all       5823      15787      0.781      0.654      0.727      0.532
             aeroplane        348        484      0.899      0.717      0.796      0.603
               bicycle        290        380      0.841      0.711       0.79      0.589
                  bird        374        629      0.782      0.596      0.688      0.476
                  boat        252        491      0.711      0.462      0.571      0.343
                bottle        369        733      0.778      0.483      0.577      0.388
                   bus        211        320      0.848      0.781      0.816      0.676
                   car        608       1173      0.821      0.662      0.761      0.551
                   cat        544        618      0.837      0.787      0.868      0.686
                 chair        642       1449      0.648      0.511      0.589      0.407
                   cow        154        347      0.739      0.697      0.723      0.523
           diningtabl

### Проверка гипотезы - использование более крупной модели YOLOv8 должно повысить точность, а увеличение количества эпох и размера входного изображения должны помочь в более тонком изучении мелких деталей.

In [ ]:
# Загрузка модели YOLOv8s
model_improved = YOLO('yolov8s.pt')

# Обучение с улучшенными параметрами
results_improved = model_improved.train(
    data='/path/to/pascal_voc/data.yaml',
    epochs=25,
    batch=20,
    imgsz=800,
    device=device
)

Ultralytics 8.3.134 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=20, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/path/to/pascal_voc/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=800, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train6, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=

train: Scanning /path/to/pascal_voc/labels/train.cache... 5717 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5717/5717 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 712.4±781.3 MB/s, size: 99.9 KB)


val: Scanning /path/to/pascal_voc/labels/val.cache... 5823 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5823/5823 [00:00<?, ?it/s]


Plotting labels to runs/detect/train6/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000417, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.00046875), 63 bias(decay=0.0)
Image sizes 800 train, 800 val
Using 2 dataloader workers
Logging results to runs/detect/train6
Starting training for 25 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      25/25      6.85G     0.6893     0.5247      1.096         49        800: 100%|██████████| 286/286 [02:39<00:00,  1.79it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 146/146 [01:07<00:00,  2.15it/s]

                   all       5823      15787      0.804       0.68      0.762      0.564



25 epochs completed in 1.617 hours.
Optimizer stripped from runs/detect/train6/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train6/weights/best.pt, 22.5MB

Validating runs/detect/train6/weights/best.pt...
Ultralytics 8.3.134 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,133,324 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 146/146 [01:09<00:00,  2.09it/s]


                   all       5823      15787      0.804       0.68      0.762      0.564
             aeroplane        348        484      0.889      0.723      0.819      0.617
               bicycle        290        380      0.871      0.708      0.808      0.599
                  bird        374        629      0.817      0.633      0.739      0.522
                  boat        252        491      0.723      0.519      0.615      0.373
                bottle        369        733      0.775      0.539      0.636      0.439
                   bus        211        320       0.88      0.778      0.859      0.715
                   car        608       1173      0.831      0.716      0.809      0.596
                   cat        544        618      0.828       0.85      0.888      0.696
                 chair        642       1449      0.696      0.521      0.612      0.434
                   cow        154        347      0.797        0.7      0.775      0.588
           diningtabl

In [ ]:
# Оценка модели на валидационной выборке
metrics = model_improved.val(data=data_yaml)


print(f"mAP50: {metrics.box.map50}") 
print(f"mAP50-95: {metrics.box.map}")  

Ultralytics 8.3.134 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,133,324 parameters, 0 gradients, 28.5 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1247.7±657.2 MB/s, size: 120.0 KB)


val: Scanning /path/to/pascal_voc/labels/val.cache... 5823 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5823/5823 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 292/292 [01:35<00:00,  3.07it/s]


                   all       5823      15787      0.803      0.681      0.762      0.564
             aeroplane        348        484      0.892      0.725       0.82      0.615
               bicycle        290        380      0.877      0.711      0.811      0.601
                  bird        374        629      0.814      0.635       0.74      0.525
                  boat        252        491      0.727      0.521      0.616      0.373
                bottle        369        733      0.776      0.539      0.637       0.44
                   bus        211        320      0.876      0.781      0.859      0.717
                   car        608       1173      0.832      0.715      0.809      0.597
                   cat        544        618      0.827      0.846      0.886      0.695
                 chair        642       1449      0.693      0.521      0.613      0.434
                   cow        154        347      0.793      0.697      0.775      0.589
           diningtabl

*Стандартный бейзлайн: mAP50: 0.7268463727488651, mAP50-95: 0.5317381328246815*

*Улучшенный бейзлайн: mAP50: 0.7621538588538207, mAP50-95: 0.5644732262758688*

Вывод: гипотеза оказалась верна.

## Имплементация собственной модели

### Полная реализация сложных моделей слишком трудоёмка, поэтому пробую упрощённую модель на основе (CNN), которая будет предсказывать один bounding box и класс объекта на изображение. 

In [14]:
import torch
import torch.nn as nn

class SimpleDetector(nn.Module):
    def __init__(self, num_classes=20):
        super(SimpleDetector, self).__init__()
        # Свёрточные слои
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 224 -> 112
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 112 -> 56
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)   # 56 -> 28
        )
        # Полносвязные слои
        self.classifier = nn.Linear(64 * 28 * 28, num_classes)  # Предсказание класса
        self.bbox_regressor = nn.Linear(64 * 28 * 28, 4)        # Предсказание bounding box

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  # Вытягиваем в вектор
        class_pred = self.classifier(x)
        bbox_pred = self.bbox_regressor(x)
        return class_pred, bbox_pred

In [ ]:
import torch
import os
from PIL import Image
import xml.etree.ElementTree as ET
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

# Список классов Pascal VOC
voc_classes = [
    'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow',
    'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'
]

class VOCDataset(Dataset):
    def __init__(self, data_dir, image_set='train', transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_set = image_set  # 'train' или 'val'


        self.images_dir = os.path.join(data_dir, 'VOCdevkit', 'VOC2012', 'JPEGImages')
        self.annotations_dir = os.path.join(data_dir, 'VOCdevkit', 'VOC2012', 'Annotations')
        self.image_set_file = os.path.join(data_dir, 'VOCdevkit', 'VOC2012', 'ImageSets', 'Main', f'{image_set}.txt')


        self.image_ids = []
        with open(self.image_set_file, 'r') as f:
            self.image_ids = [line.strip() for line in f if line.strip()]

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        img_id = self.image_ids[idx]


        img_path = os.path.join(self.images_dir, f'{img_id}.jpg')
        image = Image.open(img_path).convert('RGB')
        img_width, img_height = image.size


        ann_path = os.path.join(self.annotations_dir, f'{img_id}.xml')
        tree = ET.parse(ann_path)
        root = tree.getroot()

        # Для упрощения берём первый объект 
        obj = root.find('object')
        if obj is None:
            # Если объектов нет, возвращаем пустые данные
            class_id = 0
            bbox = torch.zeros(4)
        else:
            class_name = obj.find('name').text
            class_id = voc_classes.index(class_name) if class_name in voc_classes else 0

            # Координаты bounding box
            bndbox = obj.find('bndbox')
            xmin = float(bndbox.find('xmin').text)
            ymin = float(bndbox.find('ymin').text)
            xmax = float(bndbox.find('xmax').text)
            ymax = float(bndbox.find('ymax').text)

            # Нормализация координат [x_center, y_center, width, height]
            x_center = (xmin + xmax) / 2 / img_width
            y_center = (ymin + ymax) / 2 / img_height
            width = (xmax - xmin) / img_width
            height = (ymax - ymin) / img_height
            bbox = torch.tensor([x_center, y_center, width, height])

        # Применение трансформаций
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(class_id, dtype=torch.long), bbox

# Трансформации
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Датасет и загрузчик
dataset = VOCDataset(data_dir='/path/to/pascal_voc/', image_set='train', transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Проверка
print(f"Количество изображений в датасете: {len(dataset)}")

# Модель, потери и оптимизатор
model = SimpleDetector(num_classes=20)
criterion_class = nn.CrossEntropyLoss()
criterion_bbox = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Цикл обучения
for epoch in range(10):
    for images, labels, bboxes in dataloader:
        optimizer.zero_grad()
        class_pred, bbox_pred = model(images)
        loss_class = criterion_class(class_pred, labels)
        loss_bbox = criterion_bbox(bbox_pred, bboxes)
        loss = loss_class + loss_bbox
        loss.backward()
        optimizer.step()
    print(f"Эпоха {epoch+1}, Loss: {loss.item()}")

Количество изображений в датасете: 5717
Эпоха 1, Loss: 2.77070951461792
Эпоха 2, Loss: 2.440721035003662
Эпоха 3, Loss: 1.8826191425323486
Эпоха 4, Loss: 1.4468070268630981
Эпоха 5, Loss: 1.30607008934021
Эпоха 6, Loss: 1.1855098009109497
Эпоха 7, Loss: 0.617194652557373
Эпоха 8, Loss: 0.3829125165939331
Эпоха 9, Loss: 0.3474341928958893
Эпоха 10, Loss: 0.11439548432826996


In [20]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 41.5 MB/s eta 0:00:00


In [24]:
from torchmetrics.detection import MeanAveragePrecision

# Валидационный датасет и загрузчик
val_dataset = VOCDataset(data_dir='/path/to/pascal_voc/', image_set='val', transform=transform)
test_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Оценка
model.eval()
map_metric = MeanAveragePrecision()
with torch.no_grad():
    for images, labels, bboxes in test_dataloader:
        class_pred, bbox_pred = model(images)
        preds = [{'boxes': bbox_pred, 'scores': torch.softmax(class_pred, dim=1).max(dim=1)[0], 'labels': class_pred.argmax(dim=1)}]
        targets = [{'boxes': bboxes, 'labels': labels}]
        map_metric.update(preds, targets)
mAP = map_metric.compute()['map'].item()
print(f"mAP модели: {mAP}")

mAP модели: 0.00014734633441548795


### Выводы: имплементированная модель показала себя хуже YOLAv8n. Ограничение числа эпох, абсолютная разная сложность моделей, а также, возможно, неправильная настройка параметров сыграли свою роль.